In [30]:
import requests, os

def translate(text, src_lang='auto', to_lang='zh-CN'):
    '''
    Google翻译
    '''
    googleapis_url = 'https://translate.googleapis.com/translate_a/single'
    params = {
        'client': 'gtx',
        'sl': src_lang,
        'tl': to_lang,
        'dt': 't',
        'q': text
    }

    try:
        data = requests.get(googleapis_url, params=params)
        if data.status_code == 200:
            data = data.json()
            translations = [s[0] for s in data[0] if s[0]]
            return ' '.join(translations)
        else:
            return f'Error: {data.status_code}'
    except requests.RequestException as e:
        return f'Request Exception: {e}'
    except ValueError as e:
        return f'ValueError: {e}'

In [3]:
import numpy as np
import pandas as pd
from datetime import datetime

In [4]:
to_be_tran = pd.read_parquet('./wmj_jobtitle_translated_51946844.parquet')
save_path = './output'

In [5]:
to_be_tran1 = pd.read_csv('./nolb_skill_history_all_20230929_translate_13580161.csv')

<ipython-input-5-6e22112bc9d1>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  to_be_tran1 = pd.read_csv('./nolb_skill_history_all_20230929_translate_13580161.csv')


In [6]:
to_be_tran1.head()

,Unnamed: 0.1,Unnamed: 0,skill_raw,skill_raw_lang_id,skill_raw_lang_prob,skill_nob
0,0,1,sql,5.0,0.317322,45013088.0
1,1,5,porsche,2.0,0.244675,13899.0
2,2,7,oracle,3.0,0.375682,14487515.0
3,3,8,editorial,6.0,0.699550,5984681.0
4,4,14,linux,6.0,0.364880,25225416.0


In [13]:
file1 = to_be_tran.head()
file2 = to_be_tran1.head()
selected_cols_file1 = file1[['jobtitle', 'jobtitle_nob']]
selected_cols_file1 = selected_cols_file1.rename(columns={'jobtitle': 'A', 'jobtitle_nob': 'B'})
selected_cols_file2 = file2[['skill_raw', 'skill_nob']]
selected_cols_file2 = selected_cols_file2.rename(columns={'skill_raw': 'A', 'skill_nob': 'B'})


In [25]:
merge_two_list = pd.concat([selected_cols_file1, selected_cols_file2], ignore_index=True)
merge_two_list


,A,B
0,PROMOTORA,2602.0
1,Becario,100801.0
2,Docente,541248.0
3,Editorial Intern,93185.0
4,Server,927226.0
5,sql,45013088.0
6,porsche,13899.0
7,oracle,14487515.0
8,editorial,5984681.0
9,linux,25225416.0


In [27]:
save_path = './'
sp_list = np.array_split(merge_two_list, 3)

In [31]:
for i, sp in enumerate(sp_list):
    save_name = os.path.join(save_path, f'tran_output{i:0>2}.parquet')
    if os.path.isfile(save_name):
        continue
    print(save_name)
    print(f'start translate split_file{i} at time: {datetime.now()}')
    A_merge = sp['A'].tolist()
    all_trans = []
    for id, A in enumerate(A_merge):
        tran = translate(A)
        if id % 1000 == 0:
            print(tran)
        all_trans.append(tran)
    print(f'saved at time:{datetime.now()}')
    sp['google_translated'] = all_trans
    sp.to_parquet(save_name)
    

    

./tran_output00.parquet
start translate split_file0 at time: 2023-11-22 16:54:40.236137
检察官
saved at time:2023-11-22 16:54:40.828884
./tran_output01.parquet
start translate split_file1 at time: 2023-11-22 16:54:40.836116
服务器
saved at time:2023-11-22 16:54:41.271669
./tran_output02.parquet
start translate split_file2 at time: 2023-11-22 16:54:41.276915
甲骨文
saved at time:2023-11-22 16:54:41.723341


In [32]:
i = pd.read_parquet('./tran_output00.parquet')
i


,A,B,google_translated
0,PROMOTORA,2602.0,检察官
1,Becario,100801.0,学者
2,Docente,541248.0,老师
3,Editorial Intern,93185.0,编辑实习生
